# Historical Conditions of Central Region of the California Current

This code uses outputs from a 3km ROMS model. Within this code we will be selecting specifically for Gopher rockfish habitat

In [3]:
# import the modules you will need for this notebook
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc4
import requests
import os
import argparse
import cmocean.cm as cm
import cartopy 
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
from datetime import datetime, timedelta
import cmocean
from collections import defaultdict
import csv
import calendar

In [4]:

# --- Config ---
data_folder = r"D:\Madison"
file_list = sorted([f for f in os.listdir(data_folder) if f.endswith('.nc')])

month_indices = {"Jan":0,"Feb":1,"Mar":2,"Apr":3,"May":4,"Jun":5,"Jul":6,"Aug":7,"Sep":8,"Oct":9,"Nov":10,"Dec":11}
month_order = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

# Latitude bands (south, north, label, include_north_on_last)
# Latitude bands (south, north, label, include_north_on_last)
bands = [
    (34.4, 41.0, "ALL_41to34p4", True),  # [34.4, 41] inclusive
    (39.0, 41.0, "41to39", False),       # [39,41)
    (37.0, 39.0, "39to37", False),       # [37,39)
    (34.4, 37.0, "37to34p4", True),      # [34.4,37] inclusive top edge
]

def get_2d(ds, names):
    """Return the first 2-D variable in 'names' that exists; else None."""
    for n in names:
        if n in ds.variables:
            v = ds.variables[n]
            if v.ndim == 2:
                return np.array(v)
    return None

# Collect records separately per band
records_by_band = {label: [] for _, _, label, _ in bands}

for file_name in file_list:
    year = int(file_name.split('_')[-1].replace('.nc', ''))
    file_path = os.path.join(data_folder, file_name)

    with nc4.Dataset(file_path) as ds:
        # Expect 3-D (time, y, x)
        do_bot   = np.array(ds.variables["do_bot"])     # (T, Y, X)
        ph_bot   = np.array(ds.variables["ph_bot"])     # (T, Y, X)
        temp_bot = np.array(ds.variables["temp_bot"])   # (T, Y, X)

        # Optional chla/phyto
        chla_all = None
        if "phyto_bot" in ds.variables:
            temp = np.array(ds.variables["phyto_bot"])
            if temp.ndim == 3 and temp.shape[1:] == do_bot.shape[1:]:
                chla_all = temp

        # 2-D latitude & depth on same (Y, X) grid
        lat2d = get_2d(ds, ["lat_rho", "lat", "latitude", "y"])
        if lat2d is None:
            raise KeyError("Couldn't find a 2-D latitude variable (try 'lat_rho', 'lat', or 'latitude').")
        depth2d = get_2d(ds, ["h", "depth", "bathymetry"])
        if depth2d is None:
            # If there's no bathymetry variable, skip depth mask entirely
            depth_mask = np.ones_like(lat2d, dtype=bool)
        else:
            depth_mask = (depth2d <= 50)

    # Replace fill values
    for arr in (do_bot, ph_bot, temp_bot, chla_all):
        if isinstance(arr, np.ndarray):
            arr[arr == -1.0e+34] = np.nan

    # Build latitude band masks (non-overlapping)
    band_masks = {}
    for south, north, label, include_north in bands:
        if include_north:
            band_masks[label] = (lat2d >= south) & (lat2d <= north)   # closed on top
        else:
            band_masks[label] = (lat2d >= south) & (lat2d <  north)   # half-open

    # Loop months
    for month_name, idx in month_indices.items():
        if idx >= do_bot.shape[0]:
            continue

        # Extract monthly 2-D slices
        do_m   = do_bot[idx, :, :] * 0.032
        ph_m   = ph_bot[idx, :, :]
        temp_m = temp_bot[idx, :, :]
        chla_m = chla_all[idx, :, :] if isinstance(chla_all, np.ndarray) else None

        for _, _, label, _ in bands:
            mask = depth_mask & band_masks[label]
            if not np.any(mask):
                rec = {
                    'Year': year, 'Month': month_name,
                    'DO_avg': np.nan, 'DO_sd': np.nan,
                    'pH_avg': np.nan, 'pH_sd': np.nan,
                    'Temperature_avg': np.nan, 'Temperature_sd': np.nan,
                    'Chla_avg': np.nan, 'Chla_sd': np.nan
                }
            else:
                # Apply mask by setting outside cells to NaN, then reduce over all cells
                do_vals   = np.where(mask, do_m,   np.nan)
                ph_vals   = np.where(mask, ph_m,   np.nan)
                temp_vals = np.where(mask, temp_m, np.nan)
                chla_vals = np.where(mask, chla_m, np.nan) if chla_m is not None else None

                rec = {
                    'Year': year, 'Month': month_name,
                    'DO_avg': np.nanmean(do_vals),   'DO_sd': np.nanstd(do_vals),
                    'pH_avg': np.nanmean(ph_vals),   'pH_sd': np.nanstd(ph_vals),
                    'Temperature_avg': np.nanmean(temp_vals), 'Temperature_sd': np.nanstd(temp_vals),
                    'Chla_avg': np.nanmean(chla_vals) if chla_vals is not None else np.nan,
                    'Chla_sd': np.nanstd(chla_vals)  if chla_vals is not None else np.nan
                }

            records_by_band[label].append(rec)

# Build and save one CSV per band
for _, _, label, _ in bands:
    df = pd.DataFrame(records_by_band[label])
    df['Month'] = pd.Categorical(df['Month'], categories=month_order, ordered=True)
    df.sort_values(['Year', 'Month'], inplace=True)
    out_path = os.path.join(data_folder, f"monthly_averages_allyear_1995_2020_{label}.csv")
    df.to_csv(out_path, index=False)
    print(f"Saved: {out_path}")


Saved: D:\Madison\monthly_averages_allyear_1995_2020_ALL_41to34p4.csv
Saved: D:\Madison\monthly_averages_allyear_1995_2020_41to39.csv
Saved: D:\Madison\monthly_averages_allyear_1995_2020_39to37.csv
Saved: D:\Madison\monthly_averages_allyear_1995_2020_37to34p4.csv


### How many cells in analysis and what is the average depth of those cells?

Looking at how many cells there are and what is the average depth of the cells 

In [6]:
print(f"\n## Year: {year}")

# Annual validity per grid cell (any valid month in that year)
do_any_valid   = np.any(np.isfinite(do_bot), axis=0)
ph_any_valid   = np.any(np.isfinite(ph_bot), axis=0)
temp_any_valid = np.any(np.isfinite(temp_bot), axis=0)
chla_any_valid = np.any(np.isfinite(chla_all), axis=0) if isinstance(chla_all, np.ndarray) else None

for south, north, label, include_north in bands:

    if include_north:
        lat_mask = (lat2d >= south) & (lat2d <= north)
    else:
        lat_mask = (lat2d >= south) & (lat2d < north)

    region_mask = depth_mask & lat_mask

    # Valid masks within region
    do_valid   = region_mask & do_any_valid
    ph_valid   = region_mask & ph_any_valid
    temp_valid = region_mask & temp_any_valid
    chla_valid = region_mask & chla_any_valid if chla_any_valid is not None else None

    # Counts
    do_n   = np.sum(do_valid)
    ph_n   = np.sum(ph_valid)
    temp_n = np.sum(temp_valid)
    chla_n = np.sum(chla_valid) if chla_valid is not None else 0

    # Mean depths
    do_depth_mean   = np.nanmean(depth2d[do_valid])   if do_n   > 0 else np.nan
    ph_depth_mean   = np.nanmean(depth2d[ph_valid])   if ph_n   > 0 else np.nan
    temp_depth_mean = np.nanmean(depth2d[temp_valid]) if temp_n > 0 else np.nan
    chla_depth_mean = np.nanmean(depth2d[chla_valid]) if (chla_valid is not None and chla_n > 0) else np.nan

    # ---- Markdown-style print ----
    print(f"\n### Region: {label}")
    print(f"- Total region cells (depth ≤50m): {np.sum(region_mask)}")
    print(f"- DO cells: {do_n} | Mean depth: {do_depth_mean:.2f} m")
    print(f"- pH cells: {ph_n} | Mean depth: {ph_depth_mean:.2f} m")
    print(f"- Temperature cells: {temp_n} | Mean depth: {temp_depth_mean:.2f} m")
    if chla_any_valid is not None:
        print(f"- Chla cells: {chla_n} | Mean depth: {chla_depth_mean:.2f} m")



## Year: 2020

### Region: ALL_41to34p4
- Total region cells (depth ≤50m): 28048
- DO cells: 443 | Mean depth: 34.40 m
- pH cells: 443 | Mean depth: 34.40 m
- Temperature cells: 443 | Mean depth: 34.40 m
- Chla cells: 443 | Mean depth: 34.40 m

### Region: 41to39
- Total region cells (depth ≤50m): 11052
- DO cells: 107 | Mean depth: 36.34 m
- pH cells: 107 | Mean depth: 36.34 m
- Temperature cells: 107 | Mean depth: 36.34 m
- Chla cells: 107 | Mean depth: 36.34 m

### Region: 39to37
- Total region cells (depth ≤50m): 9044
- DO cells: 206 | Mean depth: 32.74 m
- pH cells: 206 | Mean depth: 32.74 m
- Temperature cells: 206 | Mean depth: 32.74 m
- Chla cells: 206 | Mean depth: 32.74 m

### Region: 37to34p4
- Total region cells (depth ≤50m): 7891
- DO cells: 127 | Mean depth: 35.58 m
- pH cells: 127 | Mean depth: 35.58 m
- Temperature cells: 127 | Mean depth: 35.58 m
- Chla cells: 127 | Mean depth: 35.58 m
